<a href="https://colab.research.google.com/github/Otto2005/ABP-DataCience-2025/blob/main/Digitalers_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Cargar el conjunto de datos
titanic = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# 2. Preprocesamiento y Limpieza de Datos
features = ["Pclass","Sex","Age","SibSp","Parch","Fare", "Embarked"]
label = "Survived"

# Eliminar registros con valores nulos en 'Age' y 'Embarked'
titanic = titanic.dropna(subset=["Age", "Embarked"])

# 3. Ingeniería de Características
# Crear una nueva variable 'FamilySize' y 'IsAlone'
titanic['FamilySize'] = titanic['SibSp'] + titanic['Parch'] + 1
titanic['IsAlone'] = 0
titanic.loc[titanic['FamilySize'] == 1, 'IsAlone'] = 1

# Extraer el título del nombre y mapearlo a valores numéricos
titanic['Title'] = titanic['Name'].apply(lambda name: name.split(',')[1].split('.')[0].strip())
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 5, "Col": 5, "Major": 5, "Mlle": 2, "Ms": 2, "Lady": 5, "Countess": 5, "Capt": 5, "Sir": 5, "Don": 5, "Dona": 5, "Jonkheer": 5}
titanic['Title'] = titanic['Title'].map(title_mapping).fillna(0)

# 4. Codificar variables categóricas
encoder = LabelEncoder()
titanic["Sex"] = encoder.fit_transform(titanic["Sex"])
titanic["Embarked"] = encoder.fit_transform(titanic["Embarked"])

# 5. Definir Features y Label (con las nuevas variables)
features = ["Pclass","Sex","Age","SibSp","Parch","Fare", "Embarked", "FamilySize", "IsAlone", "Title"]
X = titanic[features]
y = titanic[label]

# 6. Separar los datos entre conjuntos de entrenamiento y prueba (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 7. Optimización de Hiperparámetros con GridSearchCV
# Definir los hiperparámetros que quieres probar
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Inicializar el modelo base
gb_model = GradientBoostingClassifier(random_state=42)

# Configurar la búsqueda por cuadrícula
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entrenar el modelo con la búsqueda por cuadrícula
grid_search.fit(X_train, y_train)

# 8. Evaluación del Mejor Modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Mejores Hiperparámetros encontrados:")
print(grid_search.best_params_)
print("\nExactitud (Accuracy) del Mejor Modelo:")
print(f"{accuracy_score(y_test, y_pred):.4f}")
print("\nReporte de Clasificación del Mejor Modelo:")
print(classification_report(y_test, y_pred))

Mejores Hiperparámetros encontrados:
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}

Exactitud (Accuracy) del Mejor Modelo:
0.8037

Reporte de Clasificación del Mejor Modelo:
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       122
           1       0.82      0.70      0.75        92

    accuracy                           0.80       214
   macro avg       0.81      0.79      0.80       214
weighted avg       0.81      0.80      0.80       214

